In [1]:
import tensorflow as tf

2022-12-22 09:55:08.522349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 09:55:09.491385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 09:55:09.491426: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-22 09:55:12.277268: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
from read_data import BSData
import pandas as pd
import numpy as np
BAR_C = BSData("PWM500_BAR_CENTERED.txt")
BAR_C.read_data()
BAR_C_DF = pd.DataFrame(BAR_C.data)

In [3]:
BAR_C_DF.shape

(2695650, 10)

In [4]:
BAR_C_DF.drop('Time', axis=1, inplace=True)

,Dir,AX,AY,AZ,GX,GY,GZ,Current,TimeDiff
0,0,-0.989471,-0.040162,0.095035,2.067629,0.015259,0.457778,1.861890,0.00
1,0,-0.989471,-0.040162,0.095035,2.067629,0.015259,0.457778,1.847388,1.26
2,0,-0.989471,-0.040162,0.095035,2.067629,0.015259,0.457778,1.849805,1.01
3,0,-0.989471,-0.040162,0.095035,2.067629,0.015259,0.457778,1.853027,1.03
4,0,-0.989471,-0.040162,0.095035,2.067629,0.015259,0.457778,1.849805,1.01
...,...,...,...,...,...,...,...,...,...
2695645,0,-1.017609,0.046083,0.117496,-0.320444,0.671407,0.663778,1.858667,1.25
2695646,0,-1.017609,0.046083,0.117496,-0.320444,0.671407,0.663778,1.870752,1.25
2695647,0,-1.017609,0.046083,0.117496,-0.320444,0.671407,0.663778,1.861084,1.25
2695648,0,-1.017609,0.046083,0.117496,-0.320444,0.671407,0.663778,1.863501,1.25


In [5]:
BAR_C_Y = np.ones(BAR_C_DF.shape[0])

In [10]:
input = tf.keras.Input(shape=(9))
# x = tf.keras.layers.Flatten()(input)
x = tf.keras.layers.Dense(1024, activation='relu')(input)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(8, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(input, x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 flatten_1 (Flatten)         (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1024)              11264     
                                                                 
 dropout_5 (Dropout)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_6 (Dropout)         (None, 1024)              0         
                                                                 
 dense_9 (Dense)             (None, 1024)              1049

In [11]:
from tensorflow_addons.metrics import F1Score
model.compile(optimizer='adam', loss=tf.losses.binary_crossentropy, metrics=['accuracy', F1Score(1)])

In [12]:
checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_f1_score',
    mode='max',
    save_best_only=True
)

In [13]:
model.fit(BAR_C_DF, BAR_C_Y, validation_data=(BAR_C_DF, BAR_C_Y), epochs=1, batch_size=1024, callbacks=[model_checkpoint_callback])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2633/2633 [==============================] - 354s 134ms/step - loss: 0.1456 - accuracy: 0.9841 - f1_score: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_f1_score: 1.0000
